In [12]:
import torch
import random
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
# read all words

words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [14]:
len(words)

32033

Vocab

In [15]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0

itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [16]:
block_size = 3

def build_dataset(words):

    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] #crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y


random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
X_train, Y_train = build_dataset(words[:n1])
X_dev, Y_dev = build_dataset(words[n1:n2])
X_test, Y_test = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


Utility func to compare grads

In [17]:
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')
    


In [18]:
n_embed = 10 #dim of the char embeddings
n_hidden = 64 #num of neurons in the hidden layer of MLP

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed), generator=g)

#Layer1
W1 = torch.randn((block_size*n_embed, n_hidden), generator=g) * (5/3)/((n_embed*block_size)**0.5) #kaining init
b1 = torch.randn(n_hidden, generator=g) * 0.1 #bias is optional here since we use batchnorm after this which cancels out bias term

#Layer2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

#BatchNorm Params
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))* 0.1


parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True


4137


In [19]:
batch_size = 32
n = batch_size

#construct a minibatch
ix = torch.randint(0, X_train.shape[0], (batch_size, ), generator=g)
Xb, Yb = X_train[ix], Y_train[ix]

In [21]:
#forward pass, split into small steps

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)

#Linear Layer 1
h_pre_bn = embcat @ W1 + b1 #hidden layer preact

#Batchnorm layer
bnmeani = 1/n * h_pre_bn.sum(0, keepdim=True)
bndiff = h_pre_bn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1) * (bndiff2).sum(0, keepdim=True) #Bessel's correctn, div by n-1 not n
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
h_pre_act = bngain * bnraw + bnbias

#Non linearity
h = torch.tanh(h_pre_act) #hidden layer

#Linear Layer 2
logits = h @ W2 + b2 #output layer

#cross entropy loss
loss = F.cross_entropy(logits, Yb)

#Pytorch backward pass
for p in parameters:
    p.grad = None

for t in [emb, embcat, h_pre_bn, bnmeani, bndiff, bndiff2, bnvar, 
          bnvar_inv, bnraw, h_pre_act, h, logits, loss]:
    t.retain_grad()

loss.backward()
loss

tensor(3.3322, grad_fn=<NllLossBackward0>)

Backprop:: loss --> logits 

- loss = -logPy

- dL/dlogits = Pi if i!=y; = Pi-1 if i=y

In [23]:
#tracing the steps of forward pass
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n
cmp('dlogits', dlogits, logits)

dlogits         | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09


Batch Norm Layer: fast impl

In [ ]:
# bnmeani = 1/n * h_pre_bn.sum(0, keepdim=True)
# bndiff = h_pre_bn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1) * (bndiff2).sum(0, keepdim=True) #Bessel's correctn, div by n-1 not n
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# h_pre_act = bngain * bnraw + bnbias

In [26]:
h_pre_act = (bngain * (h_pre_bn - h_pre_bn.mean(0, keepdim=True)) / 
                torch.sqrt(h_pre_bn.var(0, keepdim=True, correction=1)) + bnbias)

In [27]:
bndiff.shape, bnvar.shape, bnraw.shape

(torch.Size([32, 64]), torch.Size([1, 64]), torch.Size([32, 64]))

- We have dh_pre_act and need to get dh_pre_bn
- dL/dbnraw = dL/dh_pre_act * bngain
- Derive the rest mathematically


In [28]:
#as in previous notebook
dh = dlogits @  W2.T

In [29]:
#as in previous notebook
dh_pre_act = (1.0 - h**2) * dh

In [30]:
#mathematically derived exp
dh_pre_bn = bngain*bnvar_inv/n * (n*dh_pre_act - dh_pre_act.sum(0)
                                     - n/(n-1)*bnraw*(dh_pre_act*bnraw).sum(0))
cmp('dh_pre_bn', dh_pre_bn, h_pre_bn)

dh_pre_bn       | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


Put it all together

In [46]:
n_embed = 10 #dim of the char embeddings
n_hidden = 64 #num of neurons in the hidden layer of MLP

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed), generator=g)

#Layer1
W1 = torch.randn((block_size*n_embed, n_hidden), generator=g) * (5/3)/((n_embed*block_size)**0.5) #kaining init
b1 = torch.randn(n_hidden, generator=g) * 0.1 #bias is optional here since we use batchnorm after this which cancels out bias term

#Layer2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

#BatchNorm Params
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))* 0.1


parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True


max_steps = 200000
batch_size=32
n = batch_size
lossi = []

with torch.no_grad(): #pytorch grads not req with this approach
    for i in range(max_steps):

        #minibatch
        ix = torch.randint(0, X_train.shape[0], (batch_size, ), generator=g)
        Xb, Yb = X_train[ix], Y_train[ix] #batch X, Y

        #forward pass
        emb = C[Xb]
        embcat = emb.view(emb.shape[0], -1)

        #Linear Layer 1
        h_pre_bn = embcat @ W1 + b1 #hidden layer preact

        #Batchnorm layer
        bnmean = h_pre_bn.mean(0, keepdim=True)
        bnvar = h_pre_bn.var(0, keepdim=True, correction=1)
        bnvar_inv = (bnvar + 1e-5)**-0.5
        bnraw = (h_pre_bn - bnmean) * bnvar_inv
        h_pre_act = bngain * bnraw + bnbias

        #Non linearity
        h = torch.tanh(h_pre_act) #hidden layer

        #Linear Layer 2
        logits = h @ W2 + b2 #output layer

        #cross entropy loss
        loss = F.cross_entropy(logits, Yb)

        #Pytorch backward pass
        for p in parameters:
            p.grad = None
        # loss.backward() #for checking

        #BackProp-------------------------------------------------------------
        dlogits = F.softmax(logits, 1)
        dlogits[range(n), Yb] -= 1
        dlogits /= n

        #2nd layer backprop
        dh = dlogits @  W2.T
        dW2 = h.T @ dlogits
        db2 = dlogits.sum(0)

        #tanh
        dh_pre_act = (1.0 - h**2) * dh

        #bactchnorm backprop
        dh_pre_bn = bngain*bnvar_inv/n * (n*dh_pre_act - dh_pre_act.sum(0)
                                            - n/(n-1)*bnraw*(dh_pre_act*bnraw).sum(0))
        dbngain = (bnraw * dh_pre_act).sum(0, keepdim=True)
        dbnbias = dh_pre_act.sum(0, keepdim=True)

        #First layer backprop
        dW1 = embcat.T @ dh_pre_bn
        db1 = dh_pre_bn.sum(0)
        dembcat = dh_pre_bn @ W1.T

        #Embedding
        demb = dembcat.view(emb.shape)
        dC = torch.zeros_like(C)
        for k in range(Xb.shape[0]):
            for j in range(Xb.shape[1]):
                ix = Xb[k, j] #ix is the token of each char
                dC[ix] += demb[k,j] #since each ix can repeat, grads will be added


        grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
        #_--------------------------------------------------------------------    

        #learning rate
        lr = 0.1 if i < 100000 else 0.01 # step learning rate decay

        #update
        for p, grad in zip(parameters, grads):
            # print(p)
            # p.data += -lr * p.grad #old way with pytorch
            p.data += -lr * grad

        #track stats
        if i % 10000 == 0:
            print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
        lossi.append(loss.log10().item())

        # if i>100:
        #     break

4137
      0/ 200000: 3.3749
  10000/ 200000: 2.4797
  20000/ 200000: 2.0459
  30000/ 200000: 2.3552
  40000/ 200000: 2.2362
  50000/ 200000: 2.2144
  60000/ 200000: 2.2377
  70000/ 200000: 2.4448
  80000/ 200000: 2.2306
  90000/ 200000: 2.4461
 100000/ 200000: 2.1030
 110000/ 200000: 1.8658
 120000/ 200000: 2.0013
 130000/ 200000: 2.5519
 140000/ 200000: 1.8117
 150000/ 200000: 2.0933
 160000/ 200000: 2.1557
 170000/ 200000: 2.3292
 180000/ 200000: 1.8799
 190000/ 200000: 2.1325


In [43]:
for p,g in zip(parameters, grads):
    cmp(str(type(p.shape)), g, p)

<class 'torch.Size'> | exact: False | approximate: True  | maxdiff: 1.862645149230957e-08
<class 'torch.Size'> | exact: False | approximate: True  | maxdiff: 2.0489096641540527e-08
<class 'torch.Size'> | exact: False | approximate: True  | maxdiff: 2.7939677238464355e-09
<class 'torch.Size'> | exact: False | approximate: True  | maxdiff: 3.725290298461914e-08
<class 'torch.Size'> | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
<class 'torch.Size'> | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
<class 'torch.Size'> | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09


In [47]:
# Calibrate the batch norm at end of training

with torch.no_grad():
    emb = C[X_train]
    embcat = emb.view(emb.shape[0], -1)
    h_pre_act = embcat @ W1 + b1
    #measure mean and stddev over entire training set
    bnmean = h_pre_act.mean(0, keepdim=True)
    bnstd = h_pre_act.std(0, keepdim=True)

In [48]:
@torch.no_grad() #disable grad tracking
def split_loss(split):
    x, y = {
        'train': (X_train, Y_train),
        'val': (X_dev, Y_dev),
        'test': (X_test, Y_test)
    }[split]

    emb = C[x] #(N, block_size, n_embed)
    embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size*n_embed)
    h_pre_act = embcat @ W1 + b1
    #batchnorm layer
    h_pre_act = bngain * (h_pre_act - bnmean)/bnstd + bnbias
    h = torch.tanh(h_pre_act)
    h = torch.tanh(h_pre_act)
    logits = h @ W2 +b2
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.147064208984375
val 2.1642048358917236
